In [1]:
%pip install pandas transformers peft accelerate sentencepiece datasets requests>=2.32.2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import torch
# from modelscope import snapshot_download, AutoModel, AutoTokenizer
# import os
# model_dir = snapshot_download('qwen/Qwen2-7B-Instruct', cache_dir='/root/autodl-tmp', revision='master')

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import PeftModel,LoraConfig, TaskType, get_peft_model
from datasets import Dataset
import torch

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
train_df = pd.read_csv('./train.csv', encoding='gb2312', encoding_errors='ignore')

In [3]:
test_df = pd.read_csv('./test.csv', encoding='gb2312', encoding_errors='ignore')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   数据编号    3500 non-null   object
 1   创作要求    2000 non-null   object
 2   评判维度    3500 non-null   object
 3   待评判内容   3475 non-null   object
 4   标注分值    3500 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 136.8+ KB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3049 entries, 0 to 3048
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   数据编号    3049 non-null   object
 1   创作要求    2049 non-null   object
 2   评判维度    3049 non-null   object
 3   待评判内容   3025 non-null   object
dtypes: object(4)
memory usage: 95.4+ KB


In [6]:
train_df[:2]

,数据编号,创作要求,评判维度,待评判内容,标注分值
0,TR0001,NaN,流畅性,美国塔科马市（47°17'N，122°28'W，是北太平洋东岸的港口城市，位于，人口约21....,3
1,TR0002,请以“针对中国人民银行及其各级分支机构，以及各类银行机构（包括政策性银行、国有商业银行、中国...,规范性,中国人民银行上海总部，各省、自治区、直辖市及计划单列市分行；各政策性银行、国有商业银行，中国...,1


In [7]:
train_df[-2:]

,数据编号,创作要求,评判维度,待评判内容,标注分值
3498,CH0499,边际贡献率和变动成本率____。 选择正确选项作答 A、反方向变化 B、同方向变化 C、同...,选择题,正确选项为A 模型回复为根据题干中的信息，我们可以推断出 反比例变化,0
3499,CH0500,"$HO_2$的沸点比$H_2S$的高,可以从____角度来进行解释。 选择正确选项作答 A...",选择题,正确选项为C 模型回复为C:氢键,1


In [8]:
test_df[:2]

,数据编号,创作要求,评判维度,待评判内容
0,TE0001,请以“描述一个初冬早晨的校园场景，展现了学生们在学业压力与日常生活小插曲中的生动画面，特别是...,规范性,那是一个潮湿的初冬的早上，学校那栋白色的教学楼像是融化在白雾里，只看见透明的或是绿色的玻璃窗...
1,TE0002,NaN,流畅性,2024年春节临近，山西博物院推出多款文创产品和多场展览推出，前来参观的游客不少吸引。\n2...


In [9]:
test_df[-2:]

,数据编号,创作要求,评判维度,待评判内容
3047,CHT1048,1948年底到1949年初，中共中央确定了一个重要的外交方针，将承认问题由新政权的单向“被承...,选择题,正确选项为D 模型回复为The correct answer is “求同存异”
3048,CHT1049,新冠肺炎疫情防控期间，“天价口罩”凸显了相关企业违反的商业伦理类型是 选择正确选项作答 A...,选择题,正确选项为D 模型回复为D.操纵价格行为


In [10]:
res = []
for i in range(len(train_df)):
    llm_item = train_df.loc[i]
    if (llm_item['评判维度']=="选择题"):
        tmp =  {
        "instruction": "如果模型回复与正确选项一致，则输出1；如果不一致，则输出0。",
        "input": f"{llm_item['创作要求']}。{llm_item['待评判内容']}",
        "output": str(llm_item['标注分值']) 
        }
        res.append(tmp)

    elif (llm_item['评判维度']=="流畅性"):
        tmp =   {
        "instruction": """下面是一个模型完成的创作内容。按照流畅性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准如下：
        1分: 非常不流畅，不具备可读性，语法错误明显，难以理解，大量拼写错误和错别字，影响阅读，表达不清晰，难以捉摸要表达的意思，每百字平均错误数超过2.5个。
        2分: 具有可读性，但较不流畅，常见语法错误多，需花费一定时间理解，一些拼写错误和错别字，阅读中断，表达较为模糊，需用一些猜测才能明白含义，每百字平均错误数介于2至2.5个。
        3分：基本流畅，存在少量语法错误，但影响较小，稍有拼写错误，但不影响阅读，主要意思表达清楚，但部分地方表述不够准确，每百字平均错误数介于1至2个。
        4分：较流畅，语法错误稀少，易读性较高，几乎无拼写错误，阅读顺畅，表达清晰、准确，容易理解，每百字平均错误数介于0.5至1个。
        5分：非常流畅，语法、拼写完美，阅读体验优秀，表达精炼、准确、得体；文句优美，行文连贯，思维严密，每百字平均错误数在0至0.5个之间。""",
        "input": f" 模型创作:{llm_item['待评判内容']}",
        "output": str(llm_item['标注分值']) 
        }
        res.append(tmp)

    else:
        tmp ={
        "instruction": """下面是一个模型，根据创作要求，完成的创作内容。按照规范性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。规范性评分标准如下：
        1分: 创作内容离题，与提示语句要求不符，格式非常不规范，杂乱无章，句子结构混乱，缺乏逻辑，每千字平均错误数超过5个。
        2分: 创作内容与提示语句要求有一定契合但覆盖不全，格式较不规范：缺乏清晰的结构，但基本逻辑仍能找到，每千字平均错误数在4至5个之间。
        3分：创作内容与提示语句要求基本契合但覆盖不全，格式一般规范，结构基本顺畅，逻辑较清晰，每千字平均错误数在2至4个之间。
        4分：创作内容与提示语句要求基本契合且基本覆盖，格式较规范，结构清晰，逻辑条理分明，每千字平均错误数在1至2个之间。
        5分：创作内容与提示语句要求完美契合，格式非常规范：结构严谨，逻辑清晰，层次分明，每千字平均错误数在0至1个之间。""",
        "input": f"创作要求:{llm_item['创作要求']}。模型创作:{llm_item['待评判内容']}",
        "output": str(llm_item['标注分值']) 
        }
        res.append(tmp)          

In [11]:
len(res)

3500

In [12]:
import json
import numpy as np

def convert_to_builtin_type(obj):
    if isinstance(obj, np.int64):  # 如果对象是 int64 类型
        return int(obj)  # 转换为内置的 int 类型
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

with open('./train_qwen2.json', mode='w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4, default=convert_to_builtin_type)

In [13]:
def process_func(example):
    MAX_LENGTH = 2048    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n您将担任评审的专家角色<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [14]:
df = pd.read_json('./train_qwen2.json')
ds = Dataset.from_pandas(df)

In [15]:
ds[:2]

{'instruction': ['下面是一个模型完成的创作内容。按照流畅性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准如下：\n        1分: 非常不流畅，不具备可读性，语法错误明显，难以理解，大量拼写错误和错别字，影响阅读，表达不清晰，难以捉摸要表达的意思，每百字平均错误数超过2.5个。\n        2分: 具有可读性，但较不流畅，常见语法错误多，需花费一定时间理解，一些拼写错误和错别字，阅读中断，表达较为模糊，需用一些猜测才能明白含义，每百字平均错误数介于2至2.5个。\n        3分：基本流畅，存在少量语法错误，但影响较小，稍有拼写错误，但不影响阅读，主要意思表达清楚，但部分地方表述不够准确，每百字平均错误数介于1至2个。\n        4分：较流畅，语法错误稀少，易读性较高，几乎无拼写错误，阅读顺畅，表达清晰、准确，容易理解，每百字平均错误数介于0.5至1个。\n        5分：非常流畅，语法、拼写完美，阅读体验优秀，表达精炼、准确、得体；文句优美，行文连贯，思维严密，每百字平均错误数在0至0.5个之间。',
  '下面是一个模型，根据创作要求，完成的创作内容。按照规范性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。规范性评分标准如下：\n        1分: 创作内容离题，与提示语句要求不符，格式非常不规范，杂乱无章，句子结构混乱，缺乏逻辑，每千字平均错误数超过5个。\n        2分: 创作内容与提示语句要求有一定契合但覆盖不全，格式较不规范：缺乏清晰的结构，但基本逻辑仍能找到，每千字平均错误数在4至5个之间。\n        3分：创作内容与提示语句要求基本契合但覆盖不全，格式一般规范，结构基本顺畅，逻辑较清晰，每千字平均错误数在2至4个之间。\n        4分：创作内容与提示语句要求基本契合且基本覆盖，格式较规范，结构清晰，逻辑条理分明，每千字平均错误数在1至2个之间。\n        5分：创作内容与提示语句要求完美契合，格式非常规范：结构严谨，逻辑清晰，层次分明，每千字平均错误数在0至1个之间。'],
 'input': [" 模型创作:美国塔科马市（47°17'N，122°28'W，是北太平洋东岸的港口城市，位于，人口约21.9万（2020年

In [16]:
model = AutoModelForCausalLM.from_pretrained('/root/share/new_models/qwen/Qwen2-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained('/root/share/new_models/qwen/Qwen2-7B-Instruct', use_fast=False, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]


In [17]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3500/3500 [01:02<00:00, 55.95 examples/s] 


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3500
})

In [18]:
tokenizer.decode(tokenized_id[0]['input_ids'])

"<|im_start|>system\n您将担任评审的专家角色<|im_end|>\n<|im_start|>user\n下面是一个模型完成的创作内容。按照流畅性评分标准给模型创作打分(只取1分、2分、3分、4分、5分其一)。流畅性评分标准如下：\n        1分: 非常不流畅，不具备可读性，语法错误明显，难以理解，大量拼写错误和错别字，影响阅读，表达不清晰，难以捉摸要表达的意思，每百字平均错误数超过2.5个。\n        2分: 具有可读性，但较不流畅，常见语法错误多，需花费一定时间理解，一些拼写错误和错别字，阅读中断，表达较为模糊，需用一些猜测才能明白含义，每百字平均错误数介于2至2.5个。\n        3分：基本流畅，存在少量语法错误，但影响较小，稍有拼写错误，但不影响阅读，主要意思表达清楚，但部分地方表述不够准确，每百字平均错误数介于1至2个。\n        4分：较流畅，语法错误稀少，易读性较高，几乎无拼写错误，阅读顺畅，表达清晰、准确，容易理解，每百字平均错误数介于0.5至1个。\n        5分：非常流畅，语法、拼写完美，阅读体验优秀，表达精炼、准确、得体；文句优美，行文连贯，思维严密，每百字平均错误数在0至0.5个之间。 模型创作:美国塔科马市（47°17'N，122°28'W，是北太平洋东岸的港口城市，位于，人口约21.9万（2020年）。该市一位名叫约克的年轻人，非常喜爱中国文化的，工作之余经常前往图书馆读书，或漫步公园游憩。1873年，横贯美洲大陆的北太平洋铁路建成。这条铁路成了分处铁路两端的约克曾祖父母缔结姻缘的纽带。他们初识时，塔科马位于铁路西端，千余人仅居住于此，到1889年人口达3.6万。图12示意塔科马市内部空间结构。<|im_end|>\n<|im_start|>assistant\n3<|endoftext|>"

In [20]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

'3<|endoftext|>'

In [21]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [22]:
model.enable_input_require_grads()

In [23]:
model.dtype

torch.bfloat16

In [24]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'k_proj', 'up_proj', 'o_proj', 'down_proj', 'q_proj', 'v_proj', 'gate_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [25]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/root/share/new_models/qwen/Qwen2-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'k_proj', 'up_proj', 'o_proj', 'down_proj', 'q_proj', 'v_proj', 'gate_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [26]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


In [27]:
args = TrainingArguments(
    output_dir="/group_share/Qwen2_7B_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=50,
    num_train_epochs=5,
    save_steps=500,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [28]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [29]:
trainer.train() #不小心中断了

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.8/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
50,1.331700
100,0.641600
150,0.568900
200,0.551800
250,0.535300
300,0.451400
350,0.474400
400,0.431200
450,0.456700
500,0.389700


/usr/local/lib/python3.8/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


KeyboardInterrupt: 

In [1]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer
 
 
def apply_lora(model_name_or_path, output_path, lora_path):
    print(f"Loading the base model from {model_name_or_path}")
    base = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=True
    )
    base_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
 
    print(f"Loading the LoRA adapter from {lora_path}")
 
    lora_model = PeftModel.from_pretrained(
        base,
        lora_path,
        torch_dtype=torch.bfloat16,
    )
 
    print("Applying the LoRA")
    model = lora_model.merge_and_unload()
 
    print(f"Saving the target model to {output_path}")
    model.save_pretrained(output_path)
    base_tokenizer.save_pretrained(output_path)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name_or_path = "/root/share/new_models/qwen/Qwen2-7B-Instruct"
output_path = "/group_share/Qwen2_7B_instruct_review"
lora_path = "/group_share/Qwen2_7B_instruct_lora/checkpoint-500"

In [3]:
apply_lora(model_name_or_path, output_path, lora_path)

Loading the base model from /root/share/new_models/qwen/Qwen2-7B-Instruct


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.01s/it]


Loading the LoRA adapter from /group_share/Qwen2_7B_instruct_lora/checkpoint-500
Applying the LoRA
Saving the target model to /group_share/Qwen2_7B_instruct_review


In [5]:
%pip install modelscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/58/bd/963488515859b641a2aa525ed20282e914dd4d3dc9471002f11b94e4a016/modelscope-1.18.0-py3-none-any.whl (5.7 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/ca/1c/89ffc63a9605b583d5df2be791a27bc1a42b7c32bab68d3c8f2f73a98cd4/urllib3-2.2.2-py3-none-any.whl (121 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/3d/09/d82fe4a34c5f0585f9ea1df090e2a71eb9bb1e469723053e1ee9f57c16f3/charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.8
    Not uninstalling urllib3 at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'urllib3'. No files were found to 

In [10]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 https://mirrors.aliyun.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 4s (766 kB/s)   33m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package git-lfs.
(Reading database ... 27010 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking git-lfs (2.9.2-1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up git-lfs 

In [ ]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = ''

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="NumberJys/Qwen2_7B_instruct_review_500", 
    model_dir="/group_share/Qwen2_7B_instruct_review" # 本地模型目录，要求目录中必须包含configuration.json
)